In [1]:
import numpy as np

# camera properties
m_camera = 0.002 # mass of camera (kg)
m_wire = 0.008 # mass of camera wire (kg)
d_camera = 3.5 # diameter of camera (mm)

# nitinol tube properties
E_nitinol = 41 # Young's modulus of nitinol (GPa) (ranges from 41-75)
density = 6.5 # density of nitinol (g/cm^3)

# design choices
max_deflection = 0.015 # maximum deflection of tubes at tip
D = 10 # dominating stiffness ratio
clearance = 0.3 # amount of clearance between tubes (mm)
tube_to_camera_ratio = 0.5 # ratio of distal tube diameter to camera diameter (for mounting camera)

In [ ]:
import matplotlib.pyplot as plt
import scipy.optimize as opt

def deflection(diameters, lengths, rho, E, m):
    g = 9.81 # gravitational acceleration (m/s^2)
    diameters = diameters * 1e-3
    lengths = lengths * 1e-3
    l1 = lengths[0]
    l2 = lengths[0] + lengths[1]
    l3 = lengths[0] + lengths[1] + lengths[2]
    rho *= 1e3
    E *= 1e9
    
    def moment_of_area(di, do):
        return np.pi/64 * (do**4 - di**4)
    
    def cross_section(di, do):
        return np.pi/4 * (do**2 - di**2)
    
    I = np.zeros(3)
    A = np.zeros(3)
    for i in range(3):
        I[i] = moment_of_area(diameters[2*i], diameters[2*i+1])
        A[i] = cross_section(diameters[2*i], diameters[2*i+1])
    
    term1 = 1/I[0] * (m*l1**3/3 + rho*A[0]*l1**4/8)
    term2 = 1/(I[0]+I[1]) * (m*(l2**3-l1**3)/3 + rho*A[0]*(l2**4-l1**4)/8 + rho*A[1]*((l2**4-l1**4)/8-(l2**3-l1**3)*l1/3+(l2**2-l1**2)*l1**2)/4)
    term3 = 1/(I[0]+I[1]+I[2]) * (m*(l3**3-l2**3)/3 + rho*A[0]*(l3**4-l2**4)/8 + rho*A[1]*((l3**4-l2**4)/8-(l3**3-l2**3)*l1/3+(l3**2-l2**2)*l1**2)/4 + rho*A[2]*((l3**4-l2**4)/8-(l3**3-l2**3)*l2/3+(l3**2-l2**2)*l2**2)/4)
    return g/E * (term1 + term2 + term3)

# pre-determined estimate for the lengths of the tubes
l1 = 50
l2 = 80
l3 = 70

test_diameters = np.linspace(0, 0.9*d1o, 1000)
deflections = np.zeros(len(test_diameters))

for i in range(len(test_diameters)):
    d1i = test_diameters[i]
    d2i = d1o + 2*clearance
    d2o = (D*(d1o**4 - d1i**4) + d2i**4)**(1/4)
    d3i = d2o + 2*clearance
    d3o = (D*(d2o**4 - d2i**4) + d3i**4)**(1/4)

    diameters = np.array([d1i, d1o, d2i, d2o, d3i, d3o])
    lengths = np.array([l1, l2, l3])
    m = m_camera + m_wire

    deflections[i] = deflection(diameters, lengths, density, E_nitinol, m)

fig, ax = plt.subplots()
ax.plot(test_diameters, deflections*1000)
ax.set_title('Tip Deflection vs. Inner Diameter of Distal Tube\n (d1o = 1.75mm, D = 10, clearance = 0.3 mm, l1 = 60 mm, l2 = 70 mm, l3 = 70 mm)', wrap=True)
ax.set_xlabel('Inner Diameter of Distal Tube (mm)')
ax.set_ylabel('Tip Deflection (mm)')
plt.show()

# find the optimal inner diameter of the distal tube
def optimize_inner_diameter(d1i, d1o, l1, l2, l3, clearance, D, rho, m, E, tolerance):
    d2i = d1o + 2*clearance
    d2o = (D*(d1o**4 - d1i**4) + d2i**4)**(1/4)
    d3i = d2o + 2*clearance
    d3o = (D*(d2o**4 - d2i**4) + d3i**4)**(1/4)

    res = np.abs(deflection(np.array([d1i, d1o, d2i, d2o, d3i, d3o]), np.array([l1, l2, l3]), rho, E, m)) * 1000 / (l1+l2+l3) - tolerance
    return res
opt_d1i = opt.bisect(optimize_inner_diameter, 0, 0.99*d1o, args=(d1o, l1, l2, l3, clearance, D, density, m_camera+m_wire, E_nitinol, max_deflection))
print('Optimal inner diameter of distal tube: {:.3f} mm'.format(opt_d1i))
print('Optimal outer diameter of distal tube: {:.3f} mm'.format(d1o))
print('Inner diameter of middle tube: {:.3f} mm'.format(d2i))
print('Outer diameter of middle tube: {:.3f} mm'.format(d2o))
print('Inner diameter of proximal tube: {:.3f} mm'.format(d3i))
print('Outer diameter of proximal tube: {:.3f} mm'.format(d3o))